In [ ]:
# Installing required libraries
!pip install llama_index
!pip install huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install "transformers[torch]" "huggingface_hub[inference]"
!pip install llama-index-llms-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 26.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_n

In [ ]:
# Importing
from llama_index.core import SimpleDirectoryReader, ServiceContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.objects import ObjectIndex
from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.llms.cohere import Cohere
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.bridge.pydantic import Field, PrivateAttr


from transformers import AutoTokenizer
# from llama_index.llms.huggingface import HuggingFaceLLM

import json
import glob

import nest_asyncio
nest_asyncio.apply()

import torch
torch.set_default_device("cpu")

In [ ]:
# Setting up the LLM

cohere_api_key = '********************' # Redacted the cohere API Key

llm = Cohere(api_key= cohere_api_key)

In [ ]:
# Using the HuggingFaceEmbeddings to create the embeddings. Could have used Cohere, but I only have limited access to Cohere

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", device='cpu')

service_context = ServiceContext.from_defaults(
    chunk_size= 1024,
    llm= llm,
    embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-5-c56324310f2c>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
# Reading all the PDFs in the directory

documents = glob.glob('*.pdf')
documents

['Boeing_787_Dreamliner.pdf',
 'Antonov_An-225_Mriya.pdf',
 'Boeing_777.pdf',
 'Airbus_A350.pdf',
 'Airbus_A320_family.pdf',
 'Boeing_747.pdf',
 'Airbus_A380.pdf',
 'Concorde.pdf',
 'Airbus_A321.pdf']

In [ ]:
# Creating the tools for each document

tools = []

for document in documents:
    documents_data = SimpleDirectoryReader(input_files= [document]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents_data)
    vector_index = VectorStoreIndex(nodes, embed_model=embed_model)

    vector_query_engine = vector_index.as_query_engine(llm = llm, similarity_top_k = 2)
    vector_query_tool = QueryEngineTool.from_defaults(name = f"vector_tool_{document}", query_engine = vector_query_engine)

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(llm = llm, response_mode = 'tree_summarize', use_async=True)
    summary_query_tool = QueryEngineTool.from_defaults(name = f"summary_tool_{document}", query_engine =summary_query_engine )

    tools.append(vector_query_tool)
    tools.append(summary_query_tool)

In [ ]:
# obj_index = ObjectIndex.from_objects(
#     tools,
#     index_cls= VectorStoreIndex, service_context=service_context
# )

# obj_retriever = obj_index.as_retriever(similarity_top_k=1)

In [ ]:
# Creating the query engine using the provided tools
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=tools,
    llm=llm
)

In [ ]:
response = query_engine.query('Tell me about Boeing 747?')

Generated 5 sub questions.
[vector_tool_Boeing_747.pdf] Q: What is the maximum speed of Boeing 747?
[summary_tool_Boeing_747.pdf] Q: What is the wingspan of Boeing 747?
[summary_tool_Airbus_A380.pdf] Q: Does Boeing 747 have a larger wingspan than Airbus A380?
[vector_tool_Boeing_747.pdf] Q: What are the different models of Boeing 747?
[summary_tool_Concorde.pdf] Q: How does the fuel efficiency of Boeing 747 compare to Concorde?
[vector_tool_Boeing_747.pdf] A: The maximum speed of the Boeing 747 is Mach 0.855 or 583 mph (939 km/h).
[vector_tool_Boeing_747.pdf] A: There are several models of the Boeing 747, a wide-body jet airliner manufactured by Boeing. Here are the different models:
- Boeing 747-100: The initial model, introduced in 1970.
- Boeing 747-200: An upgraded version of the -100, introduced in 1971, with heavier maximum takeoff weight and increased range.
- Boeing 747SP: A shortened version of the -100 introduced in 1976. SP stands for 'Special Performance'.
- Boeing 747-300:

In [ ]:
print(response.response)

The Boeing 747 is a wide-body jet airliner boasting a maximum speed of Mach 0.855, or 583 mph (939 km/h). Its wingspan spans 195 feet 8 inches (59.6 metres).

Several models of the Boeing 747 are available:

- Boeing 747-100: The original version introduced in 1970.
- Boeing 747-200: An upgraded model launched in 1971.
- Boeing 747SP: A shorter version introduced in 1976.
- Boeing 747-300: A model launched in 1983 with a stretched upper deck.
- Boeing 747-400: A heavier variant introduced in 1989.
- Boeing 747-8: A stretched version of the 747, launched in 2005.

The fuel efficiency of the Boeing 747 is significantly better than that of the Concorde.


In [ ]:
response = query_engine.query('What are some of the issues with concorde?')

Generated 2 sub questions.
[vector_tool_Concorde.pdf] Q: Search vector_tool_Concorde.pdf for issues with concorde
[summary_tool_Concorde.pdf] Q: Search summary_tool_Concorde.pdf for issues with concorde
[vector_tool_Concorde.pdf] A: I have searched the PDF for the term 'issues' and found some information that might be relevant. 

There is a section titled 'Human Factor Issues Emerge from Concorde Crash Investigation'. It discusses how an investigation into a Concorde crash revealed issues arising from human factors, including a series of errors made by the flight crew, and design features of the aircraft that increased the risks of such accidents. 

Another problem discussed in the PDF is the noise pollution caused by Concorde flights, which led to complaints from residents living near airports. This issue apparently impacted the environmental decision-making process regarding the Concorde.
[summary_tool_Concorde.pdf] A: Here are some of the issues that were faced with the Concorde pro

In [ ]:
print(response.response)

Here are some of the issues encountered by the Concorde:

- Human error from the flight crew and design features that increased the risk of accidents emerged as problems following an investigation into a Concorde crash
- It had a high angle of attack at low speeds, which caused issues during development and required a major redesign
- Considerable sales difficulties due to poor sales performance and high per-unit costs
- The sonic boom it created limited its ability to operate over land, restricting it to transoceanic flights
- High maintenance and operational costs, partly due to extensive modifications required to fix safety issues
- The 1973–74 stock market crash and the oil crisis reduced the prospects of Concorde sales due to its high fuel consumption
- Noise pollution from flights caused complaints from residents living near airports.


In [ ]:
response = query_engine.query('WHat happened to Antonov?')

Generated 2 sub questions.
[vector_tool_Antonov_An-225_Mriya.pdf] Q: Search Antonov files for information on recent developments
[summary_tool_Antonov_An-225_Mriya.pdf] Q: Search Antonov files for any mentions of issues or incidents
[vector_tool_Antonov_An-225_Mriya.pdf] A: Recent developments regarding Antonov include: 

- On February 3, 2021, Rytis Beresnevicius reported that Ukraine was considering completing a second Antonov An-225 Mriya. The article, published by Aerotime, mentioned that the fate of the second An-225 was undecided, despite the country's intention to resume the project.
- On March 26, 2020, David Kaminski-Morrow wrote about the An-225's return to flight after modernisation. The aircraft's return to service was marked by its participation in a training flight over Ukraine.
- According to a 2020 article published by Military Factory, the An-225 was classified as a heavy lift strategic long-range transport aircraft. The publication referred to it as the 'Cossack'.

I 

In [ ]:
print(response.response)

I found several events involving Antonov, a Ukrainian aircraft company:

- The Antonov An-225 aircraft was destroyed on February 27, 2022, during the Battle of Antonov Airport amidst the Russian invasion of Ukraine. The aircraft was reportedly intact two days prior, but it was ultimately destroyed in its hangar, reportedly intact. Antonov staff faced scrutiny for their handling of the situation, with some alleging that they did not do enough to prevent the loss of the aircraft. 
- Two of three Antonov officials were detained by Ukrainian authorities for allegedly preventing the Ukrainian National Guard from setting up defences at Hostomel Airport before the Russian invasion. The former head of Antonov was also charged with "official negligence".
- Ukraine considered resuming a project to complete a second Antonov An-225 Mriya, which was partially constructed during the late 1980s. The fate of this second aircraft remains undecided.
- The An-225 aircraft returned to flight after moderni

In [ ]:
response = query_engine.query('Which is the biggest aircraft among all?')

Generated 6 sub questions.
[summary_tool_Airbus_A380.pdf] Q: Can we consider the Airbus A380 as one of the biggest aircraft?
[vector_tool_Airbus_A380.pdf] Q: What are the key characteristics of Airbus A380 that make it one of the biggest aircraft?
[summary_tool_Boeing_747.pdf] Q: Are there any other aircraft that rival the Airbus A380 in terms of size?
[vector_tool_Boeing_747.pdf] Q: How do the dimensions of Boeing 747 compare to Airbus A380?
[summary_tool_Antonov_An-225_Mriya.pdf] Q: Is there any other aircraft that is bigger than Airbus A380?
[vector_tool_Antonov_An-225_Mriya.pdf] Q: What are some key specifications of Antonov An-225 that showcase its size?
[vector_tool_Boeing_747.pdf] A: The Boeing 747 and Airbus A380 are both wide-body jet airliners. Here's a comparison of their dimensions:

Boeing 747:
- Length: 43.79 m - 70.69 m (143 ft 8 in - 232 ft 3 in)
- Wingspan: 51.49 m - 59.58 m (169 ft - 195 ft 6 in)
- Height: 19.39 m - 20.79 m (63 ft 8 in - 68 ft 3 in)

Airbus A380:
- Le

In [ ]:
print(response.response)

The Antonov An-225 is bigger than the Airbus A380, which is already considered one of the biggest aircraft. The Antonov An-225 holds the title of being the heaviest aircraft ever built, with a maximum takeoff weight of 640 tonnes. It also has the largest wingspan of any operational aircraft, measuring an impressive 88.4 metres. 

Some key specifications of the An-225 include:
- Wingspan: 88.4 metres (290 feet)
- Length: 84 metres (275 feet)
- Height: 18.1 metres (59 feet)
- Cargo Hold: 1,300 cubic metres (46,000 cubic feet)
- Maximum Takeoff Weight: 640,000 kilograms
- Fuel Capacity: 375,000 litres

The Airbus A380, which is the world's largest passenger airliner, is quite a bit smaller in comparison:
- Length: 72.72 metres 
- Wingspan: 79.75 metres
- Height: 24.09 metres


In [ ]:
response = query_engine.query('Is Concorde the fastest aicraft')

Generated 2 sub questions.
[vector_tool_Concorde.pdf] Q: What is the fastest aircraft?
[summary_tool_Concorde.pdf] Q: Does the Concorde hold the record for fastest aircraft?
[vector_tool_Concorde.pdf] A: The fastest aircraft mentioned in the text is the Concorde. It could maintain a supercruise up to Mach 2.04, which equates to 2,170 km/h or 1,350 mph. The Concorde's top cruising speed was Mach 2.02 (~2,154 km/h or 1,338 mph) for optimum fuel consumption.
[summary_tool_Concorde.pdf] A: The Concorde does hold the record for being the fastest airliner or commercial passenger aircraft, with a maximum recorded speed of Mach 2.04, or around 1,354 mph. However, it is not the fastest aircraft ever built. That title goes to either the North American XB-70 Valkyrie, a prototype bomber which had a top speed of Mach 3.0+ or the SR-71 Blackbird, a reconnaissance aircraft which could reach a speed of over Mach 3.5.


In [ ]:
response.response

"Concorde is the fastest airliner or commercial passenger aircraft, with a maximum recorded speed of Mach 2.04 or around 1,354 mph. It can maintain a supercruise up to Mach 2.04 and has a top cruising speed of Mach 2.02 for optimal fuel consumption. However, it's not the fastest aircraft ever built. That distinction goes to either the North American XB-70 Valkyrie, a prototype bomber, or the SR-71 Blackbird, a reconnaissance aircraft."